# 4WD Robot Training on Google Colab (簡易版)

このノートブックは、GitHubからプロジェクトをCloneして直接学習を開始できる簡易版です。

**使用前の準備:**
1. このプロジェクトをGitHubにプッシュ
2. 下記の`GITHUB_REPO`を自分のリポジトリURLに変更
3. GPUランタイムに変更（L4推奨）

## 設定

In [ ]:
# GitHubリポジトリURL
GITHUB_REPO = "https://github.com/n01boy/isaac_4wd_rl.git"

# 学習パラメータ（T4 GPU用に最適化）
NUM_ENVS = 6            # 並列環境数（T4: 6, L4: 10, A100: 20）
MAX_ITERATIONS = 5000   # 最大イテレーション数（約10時間 on T4）

## 1. GPU確認

In [ ]:
!nvidia-smi

## 2. システム依存関係のインストール

In [ ]:
%%time
!apt-get update -qq && apt-get install -qq -y \
    git git-lfs wget \
    libglib2.0-0 libsm6 libxext6 libxrender-dev libgomp1 libglu1-mesa

print("✓ System dependencies installed")

## 3. Isaac Labのインストール（初回のみ約15分）

In [ ]:
%%time
import os

if not os.path.exists('/content/IsaacLab'):
    print("Cloning Isaac Lab...")
    !git clone -q https://github.com/isaac-sim/IsaacLab.git /content/IsaacLab
    %cd /content/IsaacLab
    print("Installing Isaac Lab (this may take 15-20 minutes)...")
    !./isaaclab.sh --install
    print("✓ Isaac Lab installed")
else:
    print("✓ Isaac Lab already installed")
    %cd /content/IsaacLab

## 4. プロジェクトのClone

In [ ]:
import os
import shutil

if os.path.exists('/content/isaac_4wd_rl'):
    print("Removing existing project...")
    shutil.rmtree('/content/isaac_4wd_rl')

print(f"Cloning project from: {GITHUB_REPO}")
!git clone -q {GITHUB_REPO} /content/isaac_4wd_rl

%cd /content/isaac_4wd_rl

# 依存関係のインストール
!pip install -q -r requirements.txt

print("✓ Project ready")

## 5. 環境変数の設定

In [ ]:
import os
import sys

os.environ['PYTHONPATH'] = f"/content/IsaacLab:{os.environ.get('PYTHONPATH', '')}"
os.environ['PROJECT_PATH'] = '/content/isaac_4wd_rl'
os.environ['HEADLESS'] = '1'

sys.path.insert(0, '/content/isaac_4wd_rl')

print("✓ Environment configured")
print(f"   Python path: {sys.path[0]}")
print(f"   Project path: {os.environ['PROJECT_PATH']}")

## 6. テスト実行（1台、100イテレーション）

まず1台で動作確認します（約5分）

In [ ]:
%%time
%cd /content/isaac_4wd_rl

!python scripts/train.py \
    --num_envs 1 \
    --headless \
    --max_iterations 100

print("\n✓ Test completed successfully!")

## 7. 本格的な学習開始

**注意**: これは数時間かかります。TensorBoardで進捗を確認できます。

In [ ]:
%%time
%cd /content/isaac_4wd_rl

print(f"Starting training with {NUM_ENVS} parallel environments...")
print(f"Max iterations: {MAX_ITERATIONS}")
print(f"Estimated time: {MAX_ITERATIONS * NUM_ENVS / 3600:.1f} hours\n")

!python scripts/train.py \
    --num_envs {NUM_ENVS} \
    --headless \
    --max_iterations {MAX_ITERATIONS}

print("\n✓ Training completed!")

## 8. TensorBoard起動（オプション）

学習中または学習後に実行して、進捗や結果を確認します。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/isaac_4wd_rl/logs

## 9. モデルをONNX形式にエクスポート

In [ ]:
import glob
import os

# 最新のチェックポイントを検索
checkpoints = glob.glob('/content/isaac_4wd_rl/logs/*/model_final.pt')

if not checkpoints:
    print("❌ No checkpoint found. Make sure training has completed.")
else:
    latest_checkpoint = sorted(checkpoints)[-1]
    print(f"Found checkpoint: {latest_checkpoint}")

    # ONNX変換
    !python scripts/export_onnx.py \
        --checkpoint {latest_checkpoint} \
        --output models/4wd_policy.onnx

    print("\n✓ Model exported to models/4wd_policy.onnx")

## 10. モデルをダウンロード

In [ ]:
from google.colab import files

onnx_path = '/content/isaac_4wd_rl/models/4wd_policy.onnx'

if os.path.exists(onnx_path):
    print("Downloading model...")
    files.download(onnx_path)
    print("\n✓ Download complete!")
    print("\nNext steps:")
    print("1. Transfer this file to your Raspberry Pi")
    print("2. Run: sudo python3 inference.py --model 4wd_policy.onnx")
else:
    print("❌ ONNX model not found. Please run the export step first.")

## 11. Google Driveにバックアップ（オプション）

In [ ]:
from google.colab import drive
import shutil

# Driveをマウント
drive.mount('/content/drive')

# バックアップ先ディレクトリ
backup_dir = '/content/drive/MyDrive/isaac_4wd_training'
os.makedirs(backup_dir, exist_ok=True)

# ログとモデルをコピー
print("Copying logs...")
if os.path.exists('/content/isaac_4wd_rl/logs'):
    shutil.copytree('/content/isaac_4wd_rl/logs', 
                    f'{backup_dir}/logs', 
                    dirs_exist_ok=True)

print("Copying models...")
if os.path.exists('/content/isaac_4wd_rl/models'):
    shutil.copytree('/content/isaac_4wd_rl/models', 
                    f'{backup_dir}/models', 
                    dirs_exist_ok=True)

print(f"\n✓ Backup completed to: {backup_dir}")

## トラブルシューティング

### エラー: "CUDA out of memory"

並列環境数を減らしてください：
```python
NUM_ENVS = 5  # 10から5に減らす
```

### エラー: "URDF file not found"

`envs/isaac_4wd_env.py`のURDFパスを確認してください。

### 学習が停止した

チェックポイントから再開できます：
```python
!python scripts/train.py \
    --num_envs 10 \
    --headless \
    --max_iterations 5000 \
    --checkpoint logs/4wd_ppo_XXXXXXXX_XXXXXX/model_XXXX.pt
```

---

**作成日**: 2026-01-04
**プロジェクト**: 4WD Robot Learning with Isaac Lab